In [1]:
!pip install streamlit pyngrok PyPDF2 python-docx sentence-transformers pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 850.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import random

# Define job roles and skills
job_data = {
    "Python Developer": ["Python", "Flask", "Django", "APIs", "SQL", "Pandas"],
    "AI/ML Engineer": ["Machine Learning", "Deep Learning", "TensorFlow", "PyTorch", "NLP", "Computer Vision"],
    "Data Scientist": ["Data Analysis", "Statistics", "Python", "R", "Scikit-learn", "SQL"],
    "Data Analyst": ["Excel", "SQL", "Power BI", "Tableau", "Data Visualization", "Python"],
    "Java Developer": ["Java", "Spring", "Hibernate", "REST APIs", "Microservices", "Maven"],
    "C++ Developer": ["C++", "STL", "OOP", "Multithreading", "Algorithms", "Data Structures"],
    "Software Engineer": ["Software Development", "Agile", "Git", "CI/CD", "Design Patterns", "Debugging"],
    "Full Stack Developer": ["HTML", "CSS", "JavaScript", "React", "Node.js", "MongoDB", "Express"],
    "Backend Developer": ["Java", "Node.js", "Django", "Databases", "APIs", "Microservices"],
    "NLP Engineer": ["NLP", "Text Classification", "SpaCy", "Transformers", "BERT", "Sentiment Analysis"],
    "Computer Vision Engineer": ["Image Processing", "OpenCV", "CNN", "YOLO", "Object Detection", "PyTorch"]
}

rows = []
for job, skills in job_data.items():
    for _ in range(10):
        # Matching pair (label=1)
        resume = f"Experienced in {', '.join(random.sample(skills, 4))}. Worked on {job.lower()} projects."
        jd = f"We are hiring a {job} with skills in {', '.join(random.sample(skills, 4))}."
        rows.append((resume, jd, 1))

        # Non-matching pair (label=0)
        other_job = random.choice([j for j in job_data if j != job])
        other_skills = job_data[other_job]
        resume = f"Skilled in {', '.join(random.sample(other_skills, 4))}. Background in {other_job.lower()}."
        rows.append((resume, jd, 0))

df = pd.DataFrame(rows, columns=["resume_text", "job_description", "label"])
df.to_csv("fine_tuning_resume_dataset.csv", index=False)
print("Dataset CSV created: fine_tuning_resume_dataset.csv")


Dataset CSV created: fine_tuning_resume_dataset.csv


In [4]:
%%writefile app.py
import streamlit as st
from sentence_transformers import SentenceTransformer, util
import PyPDF2
from docx import Document
from io import BytesIO
import pandas as pd
import time

# Load model
model = SentenceTransformer('fine_tuned_resume_model')

# UI Config
st.set_page_config(page_title="AI Resume Screener", page_icon="📋", layout="wide")

# CSS
st.markdown("""
<style>
    .header {
        background: linear-gradient(135deg, #6B73FF 0%, #000DFF 100%);
        color: white;
        padding: 2rem;
        border-radius: 10px;
        margin-bottom: 2rem;
    }
    .result-card {
        border-radius: 10px;
        padding: 1.5rem;
        margin-bottom: 1rem;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        background-color: #f9f9f9;
    }
</style>
""", unsafe_allow_html=True)

# Extractors
def extract_text_from_pdf(file):
    with BytesIO(file.read()) as f:
        reader = PyPDF2.PdfReader(f)
        return " ".join([page.extract_text() for page in reader.pages if page.extract_text()])

def extract_text_from_docx(file):
    with BytesIO(file.read()) as f:
        doc = Document(f)
        return " ".join([para.text for para in doc.paragraphs if para.text])

# Semantic Match
def analyze_resume(resume_text, job_desc):
    try:
        emb_resume = model.encode(resume_text, convert_to_tensor=True)
        emb_jd = model.encode(job_desc, convert_to_tensor=True)
        score = util.pytorch_cos_sim(emb_resume, emb_jd).item()
        score_percent = round(score * 100, 2)

        recommendation = "Strong Hire" if score_percent >= 75 else "Maybe" if score_percent >= 50 else "No"
        summary = f"""
        1. **Match Score**: {score_percent}
        2. **Recommendation**: {recommendation}
        """
        return summary, score_percent, recommendation
    except Exception as e:
        return f"Error: {str(e)}", 0, "Error"

# Main UI
def main():
    st.markdown("""
    <div class="header">
        <h1>📋 AI Resume Screener</h1>
        <p>Semantic AI Matching — No API Key Required</p>
    </div>
    """, unsafe_allow_html=True)

    uploaded_files = st.file_uploader("📁 Upload Resumes (PDF/DOCX)", accept_multiple_files=True)
    job_desc = st.text_area("📝 Paste Job Description", height=200, placeholder="Paste the job description here...")

    if st.button("🔍 Analyze Resumes", type="primary"):
        if not uploaded_files:
            st.error("Upload at least 1 resume")
        elif not job_desc.strip():
            st.error("Enter a job description")
        else:
            results = []
            progress_bar = st.progress(0)

            for i, file in enumerate(uploaded_files):
                try:
                    with st.expander(f"📄 {file.name}", expanded=True):
                        with st.spinner("Analyzing..."):
                            text = extract_text_from_pdf(file) if file.name.endswith('.pdf') else extract_text_from_docx(file)
                            analysis, score, rec = analyze_resume(text, job_desc)
                            st.markdown(f"""<div class="result-card">{analysis}</div>""", unsafe_allow_html=True)
                            results.append({"File Name": file.name, "Match Score (%)": score, "Recommendation": rec})
                    progress_bar.progress((i+1)/len(uploaded_files))
                    time.sleep(1)
                except Exception as e:
                    st.error(f"Failed to process {file.name}: {str(e)}")

            # Show table and download
            if results:
                df = pd.DataFrame(results)
                st.markdown("### 📊 Summary")
                st.dataframe(df)
                csv = df.to_csv(index=False).encode("utf-8")
                st.download_button("📥 Download Results as CSV", csv, "resume_scores.csv", "text/csv")

if __name__ == "__main__":
    main()


Writing app.py


In [5]:
# Disable wandb to avoid API key prompt
import os
os.environ["WANDB_DISABLED"] = "true"

import csv
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# Load pretrained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load data from CSV
train_examples = []
with open('fine_tuning_resume_dataset.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        train_examples.append(
            InputExample(texts=[row['resume_text'], row['job_description']], label=float(row['label']))
        )

# Prepare DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Define loss function
train_loss = losses.CosineSimilarityLoss(model)

# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100)

# Save the fine-tuned model
model.save('fine_tuned_resume_model')
print("Fine-tuned model saved in 'fine_tuned_resume_model' folder")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


Fine-tuned model saved in 'fine_tuned_resume_model' folder


In [6]:
!ngrok config add-authtoken 2ylesPLb3NX9clFkxkbctvfDLWJ_2aTdMtmLiTLz7NctF8PmV

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import ngrok
import time

ngrok.kill()
!streamlit run app.py &>/content/logs.txt &

time.sleep(5)
public_url = ngrok.connect(8501)
print("✅ Your app is live at:", public_url)

✅ Your app is live at: NgrokTunnel: "https://7661-35-232-155-235.ngrok-free.app" -> "http://localhost:8501"
